In [1]:
import sys
import keras.backend as k
sys.path.append("..")
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras_preprocessing.image import ImageDataGenerator
from toolkit import getLabelsFromDir
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.utils import class_weight 
import matplotlib
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
# setup
batch_size = 32
steps_per_epoch = int(12525/batch_size) + 1
validation_steps = int(3454/batch_size) + 1

### Model building ####

base_model = ResNet50(
            include_top=False,
            input_shape=(224, 224, 3),
            weights='imagenet')

#add a new dense layer to the end of the network inplace of the old layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

# add the outplut layer
predictions = Dense(44, activation='softmax')(x)

# create new model composed of pre-trained network and new final layers
model = Model(input=base_model.input, output=predictions)

for layer in model.layers:
    layer.trainable = True

model.summary()

# compile model
model.compile(loss='categorical_crossentropy',
            optimizer='sgd',
            metrics=['accuracy'])

train_dir = "../../images/images_genus/train/"
val_dir = "../../images/images_genus/val/"

assert(getLabelsFromDir(train_dir) == getLabelsFromDir(val_dir))
labels = getLabelsFromDir(train_dir)

train_datagen = ImageDataGenerator(rescale=1./255.)
val_datagen = ImageDataGenerator(rescale=1./255.)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    classes=labels,
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    color_mode='rgb',
                                                    target_size=(224, 224) )
val_generator = val_datagen.flow_from_directory(val_dir,
                                                    classes=labels,
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    color_mode='rgb',
                                                    target_size=(224, 224) )



/home/herri/.local/lib/python3.5/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/home/herri/.local/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Found 12525 images belonging to 44 classes.
Found 3454 images belonging to 44 classes.


In [3]:
W = class_weight.compute_class_weight('balanced', np.unique(train_generator.classes), train_generator.classes)
W

array([ 8.13311688, 14.98205742, 10.1663961 ,  9.81583072,  1.88515954,
        3.3887987 ,  5.17561983,  1.64542827,  9.18255132,  1.9104637 ,
        4.66654247,  4.12549407,  2.37215909,  0.57506887,  1.53869779,
        1.83651026,  2.10858586,  0.68427666, 10.1663961 ,  1.37516469,
        2.00464149,  5.5815508 ,  5.5815508 ,  0.2451844 ,  3.2347624 ,
        0.08777647,  3.95359848,  2.20665962,  0.68758235,  2.00464149,
        1.48259943,  0.38941052,  3.55823864,  1.44497   ,  1.21131528,
       11.38636364, 14.98205742,  1.20618259,  0.17593269,  0.6574113 ,
        1.01302168,  1.45979021, 17.79119318,  1.9104637 ])

In [4]:
#Train
history = model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=5,
                    validation_data=val_generator,
                    validation_steps=validation_steps,
                    class_weight=W)

#save
model.save("resnet50_first_weighted.h5")

Epoch 1/5
392/392 [==============================] - 226s 577ms/step - loss: 0.8826 - acc: 0.7638 - val_loss: 0.3800 - val_acc: 0.8798
Epoch 2/5
392/392 [==============================] - 222s 566ms/step - loss: 0.1489 - acc: 0.9587 - val_loss: 0.4680 - val_acc: 0.8607
Epoch 3/5
392/392 [==============================] - 221s 563ms/step - loss: 0.0589 - acc: 0.9853 - val_loss: 0.1501 - val_acc: 0.9566
Epoch 4/5
392/392 [==============================] - 221s 565ms/step - loss: 0.0242 - acc: 0.9956 - val_loss: 0.1405 - val_acc: 0.9560
Epoch 5/5
392/392 [==============================] - 222s 565ms/step - loss: 0.0135 - acc: 0.9977 - val_loss: 0.1439 - val_acc: 0.9569


In [5]:
plt.plot(history.history['acc'], label='training accuracy')
plt.plot(history.history['val_acc'], label='validation accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

NameError: name 'history' is not defined

In [ ]:
#save
model.save("resnet50_first_weighted.h5")